# Projet Velib
## Antoine Desjardins & Martin Lainée

On considère un système de vélos partagés, type Vélib, où les vélos
sont disponibles dans des stations dédiées et peuvent être empruntés
pour faire des trajets d’une station à une autre. Le but de ce projet est
de calculer les probabilités stationnaires que chaque station soit vide.

In [ ]:
pip install simpy

In [ ]:
import numpy as np
import pandas as pd
import random as rd
import simpy

On pose les données du problème


In [ ]:
# taux de depart par heure
lambada = [2.8,3.7,5.5,3.5,4.6]

# temps moyen de trajet
tau = [[0,3,5,7,7],
       [2,0,2,5,5],
       [4,2,0,3,3],
       [8,6,4,0,2],
       [7,7,5,2,0]]

# matrice de routage
p = [[0,0.22,0.32,0.2,0.26],
     [0.17,0,0.34,0.21,0.28],
     [0.19,0.26,0,0.24,0.31],
     [0.17,0.22,0.33,0,0.28],
     [0.18,0.24,0.35,0.23,0]]

On pose les conditions initiales

In [ ]:
# nombre de velos dans les stations
n_station = [20,15,17,13,18]

# nombres de velos sur la route
n_trajet = [[0,1,0,0,0],
            [1,0,1,0,0],
            [0,1,0,1,0],
            [0,0,1,0,1],
            [0,0,0,1,0]]

# Calcul Théorique

Nous proposons un modèle par colonie comme vu dans le cours. On indexe par $i$ entier les stations et par $(i→j)$ les connections entre ces stations. Pour les stations, nous avons les données suivantes:$\phi_i(n) = 0$ si $n=0$ et $\phi_i(n) = 1$ si $n>0$, avec $λ_{i,i→j} = λ_i * p_{i,j}$ . Pour les connections entre les stations, on a $\phi_{i \rightarrow j}(n)=n$ et $λ_{i \rightarrow j,j} = 60/τ_{i,j}$, avec $λ_{i \rightarrow j,k} = 0$ si $k \neq j$.


L'espace d'état dans le cas d'un seul vélo est ${\{0,1\}^n}$ avec $n$ le nombre de zones (stations + trajets entre les stations).

Dans le cas à 1 vélo, on a le générateur infinitésimal suivant :

| A               | i             | i -> j              | j        |
|-----------------|---------------|---------------------|----------|
| station i       | $- \lambda_i$ | $\lambda_i$ * $p_{ij}$ | $0$      |
| transition i->j | $0$           | $\frac{-60}{\tau_{ij}}$ | $\frac{60}{\tau_{ij}}$ |
| station j       | $0$           | $0$                 | $-\lambda_{j}$ |

La chaîne de Markov est récurrente positive sur un espace d'état fini. De plus, elle est irréductible. Donc il existe une unique mesure de probabilité $\pi$ telle que $\pi A = 0$.

$0 = (\pi A)_{i \rightarrow j} = \pi(i) *\lambda_{i}*p_{ij} - \frac{60}{\tau_{ij}} * \pi(i \rightarrow j) => \pi(i \rightarrow j) = \pi(i)*\frac{\tau_{ij}*\lambda_{i}*p_{ij}}{60}$
$0 = (\pi A)_{i} \pi(i)*(-\lambda_{i}) + Σ_{j \neq i} \frac{60}{\tau_{ij}}*\pi(i \rightarrow j) = \lambda_{i}*\pi(i) + Σ_{j \neq i}\pi{j}\lambda_{j}*p_{ij}$

Soit $\pi(i) = Σ_{j \neq i} \pi(j)*p_{ij}*\frac{λ_{j}}{λ_{i}}$

On obtient un système d'équation que l'on peut résoudre à normalisation près. En le résolvant puis en le normalisant, on obtient donc les probabilités pour chaque station que le vélo s'y trouve, desquelles on déduit immédiatement les probabilités que les stations soient vides. Avec les données du problème, on obtient le vecteur suivant: $\pi$(i est vide) = $(78,6\%,83,8\%,89,1\%,82,9\%,87,0\%)$


In [ ]:
class Station(object):
    def __init__(self,lambada,population,observation):
        self.lambada = lambada
        self.population = population
        self.maximum = population
        self.observation = observation
        self.data = []
        self.empty = [0 for i in range(observation)]
        self.pointer = 0
    
    def weight(self):
        if self.population == 0:
            return 0
        return self.lambada
    
    def observe(self):
        if self.pointer == self.observation:
            self.pointer = 0
            self.data.append(self.value())
        if self.population == 0:
          self.empty[self.pointer] = 1
        else:
          self.empty[self.pointer] = 0
        self.pointer += 1
    
    def value(self):
      V = 0
      for x in self.empty:
        V += x
      return V/self.observation
    
    def increment(self,change):
        self.population += change

class Connection(object):
    def __init__(self,tau,origin,destination,population,observation):
        self.lambada = 60/tau
        self.origin = origin
        self.destination = destination
        self.population = population
        self.observation = observation
        self.data = [0 for i in range(observation)]
        self.empty = [0 for i in range(observation)]
        self.pointer = 0
    
    def weight(self):
        return self.lambada * self.population
    
    def observe(self):
        if self.pointer == self.observation:
            self.pointer = 0
        self.data[self.pointer] = self.population
        if self.population == 0:
          self.empty[self.pointer] = 1
        else:
          self.empty[self.pointer] = 0
        self.pointer += 1
    
    def increment(self,change):
        self.population += change
    
    def value(self):
      V1,V2 = 0,0
      for x in self.data:
        V1 += x
      for x in self.empty:
        V2 += x
      return V1/self.observation,V2/self.observation
        
    
class Network(object):
    def __init__(self,Stations,Connections,Routing):
        self.Stations = Stations
        self.Connections = Connections
        self.Routing = Routing
        self.time = 0
        self.events = 0
        self.Value = []
     
    def weight(self):
        Weight = 0
        for S in self.Stations:
            Weight += S.weight()
        for C in self.Connections:
            Weight += C.weight()
        return Weight
    
    def event(self,env,freq):
        Lambada = self.weight()
        r = Lambada*rd.random()
        S = len(self.Stations) - 1
        while r >= 0 and S >= 0:
            r -= self.Stations[S].weight()
            S -= 1
        if r < 0:
            S += 1
            self.Stations[S].increment(-1)
            d = rd.random()
            s = len(self.Routing[S]) -1
            while d >= 0:
                d -= self.Routing[S][s]
                s -= 1
            s += 1
            for c in range(len(self.Connections)):
                if self.Connections[c].origin == S and self.Connections[c].destination == s:
                    self.Connections[c].increment(1)
        else:
            C = len(self.Connections) - 1
            while r >= 0 and C >= 0:
                r -= self.Connections[C].weight()
                C -= 1
            C += 1
            self.Connections[C].increment(-1)
            self.Stations[self.Connections[C].destination].increment(1)
        t = np.random.exponential(1/Lambada)
        if self.time//freq != (self.time + t)//freq:
            self.value()
            print(self.time)
        self.time += t
        self.events += 1
        for S in self.Stations:
            S.observe()
        yield env.timeout(t)
        yield env.process(self.event(env,freq))
    
    def value(self):
      V1 = []
      for S in self.Stations:
        n = len(S.data)
        V = 0
        for i in range(n):
             V += S.data[i]/n
        V1.append(V)
      self.Value.append((self.time,V1))

# Simulation

In [ ]:
def run(lambada,tau,routing,nStations,nTrajets,observation,limit=10,freq = 1,N=[]):
  if N == []:
      n = len(lambada)
      Stations = [Station(lambada[i],nStations[i],observation) for i in range(n)]
      Connections =[]
      for i in range(n):
          for j in range(n):
              if j != i:
                  Connections.append(Connection(tau[i][j],i,j,nTrajets[i][j],observation))
      N = Network(Stations,Connections,routing)
  env = simpy.Environment()
  env.process(N.event(env,freq))
  env.run(until = limit)
  return N



In [ ]:
run(lambada,tau,p,n_station,n_trajet,10000,100000)

([0,
  0,
  0,
  0,
  0,
  0.04530000000000025,
  0.08690000000000134,
  0.0902000000000014,
  0.1006000000000018,
  0.035699999999999996,
  0.08020000000000123,
  0.07000000000000094,
  0.10770000000000197,
  0.08090000000000126,
  0.07420000000000106,
  0.08430000000000135,
  0.11460000000000206,
  0.09010000000000146,
  0.09750000000000171,
  0.11200000000000206,
  0.05030000000000036,
  0.10930000000000195,
  0.12740000000000212,
  0.1659999999999994,
  0.04960000000000039],
 [53.38030000000134,
  12.351100000000077,
  6.544199999999908,
  6.8192000000000155,
  10.47589999999982,
  0.9553999999999111,
  0.9160999999999154,
  0.9156999999999155,
  0.9043999999999167,
  0.9647999999999101,
  0.9227999999999147,
  0.9313999999999137,
  0.8955999999999177,
  0.9227999999999147,
  0.9281999999999141,
  0.9183999999999152,
  0.8921999999999181,
  0.9143999999999156,
  0.9083999999999163,
  0.8936999999999179,
  0.9513999999999115,
  0.8945999999999178,
  0.8815999999999192,
  0.850599999

En utilisant le simulateur avec les données initiales fournies, nous avons fait tourner les simulations en observant la proportion totale d'évènements sur lesquels chaque station était vide. Grâce au warm start, nous avons utilisé le simulateur pour simuler 50 000 heures à la fois, jusqu'à ce que les probabilités simulées pour chaque station d'être vide reste constante au dixième de pourcent près. Au bout de 250 000 heures de simulation, c'était bien le cas. Nous obtenons alors le vecteur $π$ suivant: $\pi$(i est vide) = $(0,7\%,4,3\%,14,0\%,4,1\%,10,2\%)$